In [1]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.modflow.types.discretization.spatial.GridCells import GridCells
from morpheus.modflow.types.discretization import SpatialDiscretization
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit, Grid, Crs
from morpheus.modflow.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
x_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=GridCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

## Setup Time Discretization

In [3]:
from morpheus.modflow.types.discretization.time.TimeUnit import TimeUnit
from morpheus.modflow.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.modflow.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

# Setup Modflow Model

In [4]:
from morpheus.modflow.types.ModflowModel import ModflowModel

modflow_model = ModflowModel.new()
modflow_model = modflow_model.with_updated_time_discretization(time_discretization)
modflow_model = modflow_model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [5]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with 1 observation

In [6]:
from morpheus.modflow.types.boundaries.ConstantHeadObservation import ObservationId, HeadValue, ConstantHeadRawDataItem
from morpheus.modflow.types.geometry import Point
from morpheus.modflow.types.boundaries.Boundary import BoundaryName
from morpheus.modflow.types.soil_model import LayerId
from morpheus.modflow.types.boundaries.Boundary import ConstantHeadBoundary, ConstantHeadObservation

layers_list = [LayerId.new()]
constant_head = ConstantHeadBoundary.from_geometry(
  name=BoundaryName('constant_head'),
  geometry=LineString([
    (13.9223, 50.9662),
    (13.9250, 50.96615),
  ]),
  grid=grid,
  affected_layers=[modflow_model.soil_model.layers[0].id],
  observations=[
    ConstantHeadObservation(
      observation_id=ObservationId.new(),
      geometry=Point((13.9223, 50.9662)),
      raw_data=[
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          head=HeadValue.from_float(100),
        ),
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          head=HeadValue.from_float(120),
        ),
      ]
    )
  ]
)

m.add_child(folium.GeoJson(constant_head.as_geojson()))
m.add_child(folium.GeoJson(constant_head.affected_cells.as_geojson(grid).as_geojson()))
for constant_head.observation in constant_head.observations:
  m.add_child(folium.GeoJson(constant_head.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:
from morpheus.modflow.types.boundaries.Boundary import BoundaryCollection

boundary_collection = BoundaryCollection.new()
boundary_collection.add_boundary(constant_head)

modflow_model = modflow_model.with_updated_boundaries(boundaries=boundary_collection)

## Calculate

In [8]:
from morpheus.modflow.infrastructure.calculation.engines.modflow_2005.types.Mf2005CalculationEngineSettings import \
  Mf2005CalculationEngineSettings
from morpheus.modflow.types.calculation.CalculationProfile import CalculationProfile, CalculationType
from morpheus.modflow.types.calculation.Calculation import Calculation


calculation_engine_settings = Mf2005CalculationEngineSettings.default()
calculation_profile = CalculationProfile.new(CalculationType.MF2005)
calculation = Calculation.new(modflow_model=modflow_model, calculation_profile=calculation_profile)

In [9]:
calculation = Calculation.from_dict(calculation.to_dict())
calculation.calculation_id

CalculationId(value='69134951-d538-4403-9e6e-920258f805fd')

In [10]:
from morpheus.modflow.infrastructure.calculation.services.CalculationService import CalculationService

calculation_service = CalculationService.from_calculation(calculation)
calculation_service.calculate()

FloPy is using the following executable to run the model: ../../../../../../src/backend/.venv/bin/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: 7ec4a707-4ab4-414f-a71c-182e3269efe7.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/01/15 22:26:02

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/01/15 22:26:02
 Elapsed run time:  0.014 Seconds

  Normal termination of simulation


In [11]:
calculation_service.get_result()

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='Calculation finished successfully', files=['7ec4a707-4ab4-414f-a71c-182e3269efe7.list', '7ec4a707-4ab4-414f-a71c-182e3269efe7.pcg', '7ec4a707-4ab4-414f-a71c-182e3269efe7.hds', '7ec4a707-4ab4-414f-a71c-182e3269efe7.oc', '7ec4a707-4ab4-414f-a71c-182e3269efe7.dis', '7ec4a707-4ab4-414f-a71c-182e3269efe7.chd', '7ec4a707-4ab4-414f-a71c-182e3269efe7.lpf', '7ec4a707-4ab4-414f-a71c-182e3269efe7.bas', '7ec4a707-4ab4-414f-a71c-182e3269efe7.nam'], head_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=1, number_of_observations=0), drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=0, number_of_observations=0), concentration_results=None)

In [12]:
head = calculation_service.read_head(0, 0)

In [13]:
import simplejson as json

json = json.dumps(head, ignore_nan=True)
json

'[[113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, null], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, null], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [null, 113.33300

In [14]:
calculation_service.read_drawdown(0, 0)

[]

In [15]:
calculation_service.read_budget(0, False)

{'STORAGE_IN': 0.0,
 'CONSTANT_HEAD_IN': 0.0,
 'TOTAL_IN': 0.0,
 'STORAGE_OUT': -0.0,
 'CONSTANT_HEAD_OUT': -0.0018196,
 'TOTAL_OUT': -0.0018196,
 'IN-OUT': -0.0018196,
 'PERCENT_DISCREPANCY': -200.0,
 'tslen': 365.0}

In [16]:
calculation_service.read_budget(0, True)

{'STORAGE_IN': 0.0,
 'CONSTANT_HEAD_IN': 0.0,
 'TOTAL_IN': 0.0,
 'STORAGE_OUT': -0.0,
 'CONSTANT_HEAD_OUT': -4.9852e-06,
 'TOTAL_OUT': -4.9852e-06,
 'IN-OUT': -4.9852e-06,
 'PERCENT_DISCREPANCY': -200.0,
 'tslen': 365.0}

In [17]:
calculation_service.get_result()

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='Calculation finished successfully', files=['7ec4a707-4ab4-414f-a71c-182e3269efe7.list', '7ec4a707-4ab4-414f-a71c-182e3269efe7.pcg', '7ec4a707-4ab4-414f-a71c-182e3269efe7.hds', '7ec4a707-4ab4-414f-a71c-182e3269efe7.oc', '7ec4a707-4ab4-414f-a71c-182e3269efe7.dis', '7ec4a707-4ab4-414f-a71c-182e3269efe7.chd', '7ec4a707-4ab4-414f-a71c-182e3269efe7.lpf', '7ec4a707-4ab4-414f-a71c-182e3269efe7.bas', '7ec4a707-4ab4-414f-a71c-182e3269efe7.nam'], head_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=1, number_of_observations=0), drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=0, number_of_observations=0), concentration_results=None)